In [8]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# List the contents of the target directory
base_dir = "/content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data"
print("Folders in Test_data:")
print(os.listdir(base_dir))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folders in Test_data:
['shap_e', 'point_e', 'DreamFusion', 'magic3d_coarse', 'magic3d_refine', 'Our_method', '.ipynb_checkpoints']


In [7]:
import shutil

shutil.rmtree('/content/sample_data')

In [9]:
# Function to list file types in each folder
def inspect_file_types(base_dir):
    folder_types = {}
    for model_folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, model_folder)
        if os.path.isdir(folder_path):
            file_types = [os.path.splitext(file)[1] for file in os.listdir(folder_path)]
            folder_types[model_folder] = set(file_types)
    return folder_types

# Inspect file types
file_types = inspect_file_types(base_dir)
print("File types in each folder:")
print(file_types)

File types in each folder:
{'shap_e': {'.ply'}, 'point_e': {'.ply'}, 'DreamFusion': {'.mp4'}, 'magic3d_coarse': {'.mp4'}, 'magic3d_refine': {'', '.mp4'}, 'Our_method': {'.mp4'}, '.ipynb_checkpoints': set()}


In [ ]:
%%capture
!pip install open3d
!pip install trimesh pyrender

In [ ]:
%%capture
# Install necessary libraries and tools
!apt-get update -qq && apt-get install -y xvfb
!pip install pyvista imageio pytorch-fid transformers torch
!pip install git+https://github.com/openai/CLIP.git

# Start xvfb for headless rendering
import pyvista as pv
pv.start_xvfb()

# Verify pyvista installation
print("Pyvista and XVFB initialized successfully!")

In [13]:
import os
import pyvista as pv

# Input directories
base_dir = "/content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data"
folders_to_process = ["shap_e", "point_e"]

# Output directory for rendered images
output_folder = "/content/rendered_images"
os.makedirs(output_folder, exist_ok=True)

# Define viewpoints for front and back
viewpoints = {
    "front": (0, 0),  # Front
    "back": (0, 180)  # Back
}

def render_two_views(ply_path, output_folder, viewpoints):
    """
    Render front and back views of a .ply file and save as images.
    """
    model_name = os.path.splitext(os.path.basename(ply_path))[0]
    mesh = pv.read(ply_path)

    for view_name, (elev, azim) in viewpoints.items():
        plotter = pv.Plotter(off_screen=True)
        plotter.add_mesh(mesh)
        plotter.camera_position = 'xy'
        plotter.camera.elevation = elev
        plotter.camera.azimuth = azim
        img_path = os.path.join(output_folder, f"{model_name}_{view_name}.png")
        plotter.show(screenshot=img_path)
        plotter.close()

def process_ply_folder(input_folder, output_folder):
    """
    Process all .ply files in a folder and render front and back views for each.
    """
    for file in os.listdir(input_folder):
        if file.endswith(".ply"):
            ply_path = os.path.join(input_folder, file)
            try:
                print(f"Processing: {ply_path}")
                render_two_views(ply_path, output_folder, viewpoints)
            except Exception as e:
                print(f"Error rendering {ply_path}: {e}")

# Process all folders
for folder in folders_to_process:
    input_folder = os.path.join(base_dir, folder)
    output_folder_for_folder = os.path.join(output_folder, folder)
    os.makedirs(output_folder_for_folder, exist_ok=True)
    process_ply_folder(input_folder, output_folder_for_folder)

print(f"Rendered front and back views saved in {output_folder}")

Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/A_green_cupcake.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/A_film_camera.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/A_dinner_plate.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/A_steel_thermos.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/A_guitar_pick.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/A_ski_pair.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/An_old_wheel.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/shap_e/A_gumball_machine.ply
Processing: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION

In [14]:
import cv2
import os

# Input directories for all models (including DreamFusion)
base_dir = "/content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data"
folders_to_process = ["DreamFusion","magic3d_coarse","magic3d_refine","Our_method"]  # Add other models here if needed

# Output directory for extracted frames
output_base_folder = "/content/rendered_images"
os.makedirs(output_base_folder, exist_ok=True)

def extract_two_frames_as_front_back(video_path, output_folder):
    """
    Extract two frames (first and midpoint) from a video and save as front and back images.
    """
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_positions = [0, total_frames // 2]

    for i, frame_pos in enumerate(frame_positions):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
        ret, frame = cap.read()
        if ret:
            frame_filename = os.path.join(output_folder, f"{video_name}_view_{'front' if i == 0 else 'back'}.png")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved frame: {frame_filename}")
    cap.release()

def process_mp4_folder(input_folder, output_folder):
    """
    Process all .mp4 files in a folder and extract two frames (front and back) from each.
    """
    for file in os.listdir(input_folder):
        if file.endswith(".mp4"):
            video_path = os.path.join(input_folder, file)
            print(f"Processing video: {video_path}")
            extract_two_frames_as_front_back(video_path, output_folder)

# Process all folders specified in folders_to_process
for folder in folders_to_process:
    input_folder = os.path.join(base_dir, folder)
    output_folder_for_folder = os.path.join(output_base_folder, folder)
    os.makedirs(output_folder_for_folder, exist_ok=True)
    process_mp4_folder(input_folder, output_folder_for_folder)

print(f"Front and back frames saved in {output_base_folder}")

Streaming output truncated to the last 5000 lines.
Saved frame: /content/rendered_images/DreamFusion/A_toy_castle_view_back.png
Processing video: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/DreamFusion/A_garden_sprinkler.mp4
Saved frame: /content/rendered_images/DreamFusion/A_garden_sprinkler_view_front.png
Saved frame: /content/rendered_images/DreamFusion/A_garden_sprinkler_view_back.png
Processing video: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/DreamFusion/An_old_barrel.mp4
Saved frame: /content/rendered_images/DreamFusion/An_old_barrel_view_front.png
Saved frame: /content/rendered_images/DreamFusion/An_old_barrel_view_back.png
Processing video: /content/drive/MyDrive/CAP_6411_COMPUTER_VISION_SYSTEMS/final_project/Test_data/DreamFusion/A_bath_sponge.mp4
Saved frame: /content/rendered_images/DreamFusion/A_bath_sponge_view_front.png
Saved frame: /content/rendered_images/DreamFusion/A_bath_sponge_view_back.png
Pr

In [16]:
import os

def count_images_in_folder(folder_path):
  """Counts the number of image files in a given folder."""
  if not os.path.exists(folder_path):
    return 0
  image_count = 0
  for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
      image_count += 1
  return image_count


dreamfusion_count = count_images_in_folder("/content/rendered_images/DreamFusion")
point_e_count = count_images_in_folder("/content/rendered_images/point_e")
shap_e_count = count_images_in_folder("/content/rendered_images/shap_e")
magic3d_coarse_count = count_images_in_folder("/content/rendered_images/magic3d_coarse")
magic3d_refine_count = count_images_in_folder("/content/rendered_images/magic3d_refine")
Our_method_count = count_images_in_folder("/content/rendered_images/Our_method")




print(f"Number of images in DreamFusion: {dreamfusion_count}")
print(f"Number of images in point_e: {point_e_count}")
print(f"Number of images in shap_e: {shap_e_count}")
print(f"Number of images in magic3d_coarse: {magic3d_coarse_count}")
print(f"Number of images in magic3d_refine: {magic3d_refine_count}")
print(f"Number of images in Our_method: {Our_method_count}")

Number of images in DreamFusion: 884
Number of images in point_e: 884
Number of images in shap_e: 884
Number of images in magic3d_coarse: 814
Number of images in magic3d_refine: 794
Number of images in Our_method: 884


In [ ]:
import pandas as pd
import clip
import torch
from PIL import Image
from pytorch_fid import fid_score
import os

# Load CLIP model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)

def calculate_clip_score_for_file(image_path, prompt):
    """
    Calculate the CLIP score for a single image compared to a text prompt.
    """
    text_input = clip.tokenize([prompt]).to(device)
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_input)
        similarity = (image_features @ text_features.T).item()
    return similarity

def calculate_clip_score_for_folder(image_folder):
    """
    Calculate the average CLIP score for all images in a folder.
    """
    total_score = 0
    count = 0
    for image_file in os.listdir(image_folder):
        if image_file.endswith(".png"):
            prompt = os.path.splitext(image_file)[0].replace("_", " ")
            image_path = os.path.join(image_folder, image_file)
            score = calculate_clip_score_for_file(image_path, prompt)
            total_score += score
            count += 1
    return total_score / count if count > 0 else 0

def calculate_fid(folder1, folder2):
    """
    Calculate the FID score between two folders of rendered images.
    """
    return fid_score.calculate_fid_given_paths(
        [folder1, folder2],
        batch_size=50,
        device=device,
        dims=2048
    )

# Directories for rendered images
folders_to_process = ["shap_e", "point_e", "DreamFusion","magic3d_coarse","magic3d_refine","Our_method"]
base_output_path = "/content/rendered_images"

# Calculate CLIP Scores
clip_scores = {}
for folder in folders_to_process:
    folder_path = os.path.join(base_output_path, folder)
    clip_scores[folder] = calculate_clip_score_for_folder(folder_path)

# Calculate FID Scores
fid_scores = {
    "Shap-E vs. Point-E": calculate_fid(
        os.path.join(base_output_path, "shap_e"),
        os.path.join(base_output_path, "point_e")
    ),
    "Shap-E vs. DreamFusion": calculate_fid(
        os.path.join(base_output_path, "shap_e"),
        os.path.join(base_output_path, "DreamFusion")
    ),
    "Point-E vs. DreamFusion": calculate_fid(
        os.path.join(base_output_path, "point_e"),
        os.path.join(base_output_path, "DreamFusion")
    )
}

# Create a DataFrame for the results
results = pd.DataFrame({
    "Model": folders_to_process + list(fid_scores.keys()),
    "CLIP Score (Avg)": [clip_scores.get(folder, None) for folder in folders_to_process] + [None] * len(fid_scores),
    "FID Score": [None] * len(folders_to_process) + list(fid_scores.values())
})

# Display the results DataFrame
print("Model Evaluation Results:")
print(results.to_string(index=False))

100%|██████████| 18/18 [00:07<00:00,  2.52it/s]


Model Evaluation Results:
                  Model  CLIP Score (Avg)  FID Score
                 shap_e         20.044365        NaN
                point_e         19.512311        NaN
            DreamFusion         26.935520        NaN
     Shap-E vs. Point-E               NaN 185.898524
 Shap-E vs. DreamFusion               NaN 156.817814
Point-E vs. DreamFusion               NaN 237.882431


In [17]:
import pandas as pd
import clip
import torch
from PIL import Image
from pytorch_fid import fid_score
import os

# Load CLIP model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)

def calculate_clip_score_for_file(image_path, prompt):
    """
    Calculate the CLIP score for a single image compared to a text prompt.
    """
    text_input = clip.tokenize([prompt]).to(device)
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_input)
        similarity = (image_features @ text_features.T).item()
    return similarity

def calculate_clip_score_for_folder(image_folder):
    """
    Calculate the average CLIP score for all images in a folder.
    """
    total_score = 0
    count = 0
    for image_file in os.listdir(image_folder):
        if image_file.endswith(".png"):
            prompt = os.path.splitext(image_file)[0].replace("_", " ")
            image_path = os.path.join(image_folder, image_file)
            score = calculate_clip_score_for_file(image_path, prompt)
            total_score += score
            count += 1
    return total_score / count if count > 0 else 0

def calculate_fid(folder1, folder2):
    """
    Calculate the FID score between two folders of rendered images.
    """
    return fid_score.calculate_fid_given_paths(
        [folder1, folder2],
        batch_size=50,
        device=device,
        dims=2048
    )

# Directories for rendered images
folders_to_process = ["shap_e", "point_e", "DreamFusion", "magic3d_coarse", "magic3d_refine", "Our_method"]
base_output_path = "/content/rendered_images"

# Calculate CLIP Scores
clip_scores = {}
for folder in folders_to_process:
    folder_path = os.path.join(base_output_path, folder)
    clip_scores[folder] = calculate_clip_score_for_folder(folder_path)

# Calculate FID Scores
fid_scores = {}

# Generate FID scores for all possible pairs of folders
for i in range(len(folders_to_process)):
    for j in range(i + 1, len(folders_to_process)):
        folder1 = folders_to_process[i]
        folder2 = folders_to_process[j]
        fid_scores[f"{folder1} vs. {folder2}"] = calculate_fid(
            os.path.join(base_output_path, folder1),
            os.path.join(base_output_path, folder2)
        )

# Create a DataFrame for the results
results = pd.DataFrame({
    "Model": folders_to_process + list(fid_scores.keys()),
    "CLIP Score (Avg)": [clip_scores.get(folder, None) for folder in folders_to_process] + [None] * len(fid_scores),
    "FID Score": [None] * len(folders_to_process) + list(fid_scores.values())
})

# Display the results DataFrame
print("Model Evaluation Results:")
print(results.to_string(index=False))

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 96.3MiB/s]
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 367MB/s]
100%|██████████| 18/18 [00:08<00:00,  2.22it/s]


Model Evaluation Results:
                            Model  CLIP Score (Avg)  FID Score
                           shap_e         20.044542        NaN
                          point_e         19.512488        NaN
                      DreamFusion         26.935591        NaN
                   magic3d_coarse         27.060494        NaN
                   magic3d_refine         26.899146        NaN
                       Our_method         26.764644        NaN
               shap_e vs. point_e               NaN 185.898524
           shap_e vs. DreamFusion               NaN 156.817814
        shap_e vs. magic3d_coarse               NaN 161.992184
        shap_e vs. magic3d_refine               NaN 143.024820
            shap_e vs. Our_method               NaN 134.569690
          point_e vs. DreamFusion               NaN 237.882431
       point_e vs. magic3d_coarse               NaN 241.370946
       point_e vs. magic3d_refine               NaN 210.389283
           point_e vs. Our_me